Seq2Seq(Sequence-to-Sequence) 문제 정의와 해결 전략 이해  
기본 순환신경망(RNN) 한계와 LSTM 개선 아이디어 파악  
Encoder-Decoder 구조의 정보 흐름(컨텍스트 벡터/숨겨진 상태) 이해  
Teacher Forcing 기법의 학습 안정화 역할 및 Trade-off 이해  
Softmax와 CrossEntropy 손실의 수식 및 직관적 해석  
Inference 단계(그리디 vs 빔 서치)의 실전 활용 전략  
전체 파이프라인을 하나의 흐름으로 연결하여 종합 정리  

<span style="font-size:12px;">

Seq2Seq(Sequence-to-Sequence)
- 하나의 시퀀스(입력 문장) 를 다른 시퀀스(출력 문장) 로 바꾸는 모델
- 출현 배경
    - 입력과 출력의 길이가 다를 수 있는 문제(번역, 요약, 챗봇)  
    - 고정크기의 피처벡터는 순서정보와 문맥관계를 충분히 반영 못함  
    - 단순 분류모델은 시퀀스 간 종속적 생성(토큰별 점직적 예측)에 부적합  
- 예시
    - 영어를 한국어로 번역할 경우
        - 입력 시퀀스 : "I love you"
        - 출력 시퀀스 : "나는 너를 사랑해"  
        - 실제 사용 예시 : 번역, 대화시스템, 문서요약, 코드자동생성  
<br>
- 동작 원리
    - `인코더 (Encoder)` : 입력 문장을 한 단어씩 받아서 전체 의미를 하나의 벡터로 요약 -> 마지막/전체 Hidden state 집약
        - 입력: I → love → you
        - 출력: 문장 의미 벡터 (context vector)
    - `context(요약표현)` : 인코더의 정보를 압축
    - `디코더 (Decoder)`  : 인코더에서 전달받은 context vector를 이용해서 출력 문장을 한 단어씩 생성
        - 입력: context vector
        - 출력: 나는 → 너를 → 사랑해
    - `종료` : 특별한 EOS 토큰이 나올때까지



In [ ]:
입력 문장: I love you
──────────────────────────────
Encoder RNN:
   I     →     love     →     you
   ↓             ↓              ↓
 [h1] → [h2] → [h3] = context vector

──────────────────────────────
Decoder RNN:
   context vector → "나는" → "너를" → "사랑해"

<span style="font-size:12px;">

인코더-디코더 구조
1. 개념 정의
Encoder는 입력 시퀀스를 내부 상태로 변환하는 모듈, Decoder는 그 상태(컨텍스트)를 참조하며 출력 시퀀스를 생성하는 모듈

2. 왜 필요한가

- 기능 분리: 이해(Encoding)와 생성(Decoding)을 모듈화하여 확장성 확보.
- 다양한 입력/출력 도메인(텍스트→텍스트, 텍스트→태그 등)에 쉽게 적용.
- 나중에 Attention 같은 추가 요소 삽입이 용이 (기본 Encoder-Decoder가 뼈대).

3. 동작 원리

- Encoder 단계:
    - 입력 토큰 x₁, x₂, …, x_T
    - 순환구조(RNN/LSTM/GRU)가 $h_t = f(x_t, h_{t-1})$ 계산
    - 최종 $h_T$ 혹은 전체 ${h₁…h_T}$ 를 요약(기본 Seq2Seq는 $h_T$ 단일)

- Decoder 단계:
    - 초기 Hidden State: $h_T$ (Encoder 마지막 상태)
    - 반복: $y_t = Softmax(W·h_t^dec)$
    - 다음 입력: (Teacher Forcing 시 GT 토큰, 아니면 이전 예측)

4. 실제 사용 예시

| 작업 | Encoder 입력 | Decoder 출력 |
|------|---------------|---------------|
| 번역 | 영어 문장 토큰 | 한국어 문장 토큰 |
| 질의응답 | 질문 토큰 | 답변 토큰 |
| 요약 | 긴 문서 토큰 | 요약 문장 토큰 |
| 형태소 분석 | 음절 시퀀스 | 품사 태그 시퀀스 |